# COMPRENDIENDO LOS DATA SETS

In [2]:
import pandas as pd

# Cargar solo las primeras filas para inspección
file_path = "CirrusLink_100.csv"
df = pd.read_csv(file_path, nrows=5)

df.head()


,INSERTED_AT,MSG
0,1748235601721,"{\n ""metrics"": [\n {\n ""dataType"": ""T..."
1,1748235601729,"{\n ""metrics"": [\n {\n ""dataType"": ""T..."
2,1748235601731,"{\n ""metrics"": [\n {\n ""dataType"": ""T..."
3,1748235613882,"{\n ""metrics"": [\n {\n ""dataType"": ""T..."
4,1748235613883,"{\n ""metrics"": [\n {\n ""dataType"": ""T..."


In [3]:
import json

# Cargar solo unas filas para análisis
file_path = "CirrusLink_100.csv"
df = pd.read_csv(file_path, nrows=5)

# Tomar la primera fila como ejemplo
msg_str = df.loc[1, "MSG"]

# Cargar el string como dict de Python
msg_dict = json.loads(msg_str)

# Ver la estructura general (clave de primer nivel)
print(msg_dict.keys())


dict_keys(['metrics', 'seq', 'timestamp'])


In [4]:
# Ver top level
print("Top-level keys:", msg_dict.keys())

# Acceder a los "metrics"
metrics = msg_dict["metrics"]
print("Cantidad de elementos en metrics:", len(metrics))

# Ver el primero de los elementos de "metrics"
first_metric = metrics[2]
print("Primer metric:", first_metric)

# Acceder a los "value" del primer metric
value = first_metric["value"]
print("Keys dentro de value:", value.keys())

# Acceder a los sub-metrics
sub_metrics = value["metrics"]
print("Sub-metrics:", sub_metrics)


Top-level keys: dict_keys(['metrics', 'seq', 'timestamp'])
Cantidad de elementos en metrics: 20
Primer metric: {'dataType': 'Template', 'name': 'G201L/Snowflake', 'timestamp': 1748235600665, 'value': {'isDefinition': False, 'metrics': [{'dataType': 'DateTime', 'name': 'Timestamp', 'timestamp': 1748235600663, 'value': 1748235600663}], 'reference': 'Snowflake_DIA_NE_XT4_Model', 'version': ''}}
Keys dentro de value: dict_keys(['isDefinition', 'metrics', 'reference', 'version'])
Sub-metrics: [{'dataType': 'DateTime', 'name': 'Timestamp', 'timestamp': 1748235600663, 'value': 1748235600663}]


In [17]:
# Cargar ambos archivos completos
df_cirrus = pd.read_csv("CirrusLink_100.csv")
df_cedalo = pd.read_csv("Cedalo_100.csv")

# Elegimos una fila de ejemplo
i = 0  # Puedes iterar esto luego para comparar todas

# --- Cirrus ---
cirrus_msg = json.loads(df_cirrus.loc[i, "MSG"])
cirrus_first_metric = cirrus_msg["metrics"][1]
cirrus_name = cirrus_first_metric["name"]
cirrus_submetrics = cirrus_first_metric["value"]["metrics"]

# --- Cedalo ---
cedalo_payload = json.loads(df_cedalo.loc[2, "PAYLOAD_JSON"])
cedalo_first_element = cedalo_payload[1]
cedalo_name = cedalo_first_element["name"]
cedalo_submetrics = cedalo_first_element["value"]["metrics"]

# --- Comparación ---
names_equal = cirrus_name == cedalo_name
submetrics_equal = cirrus_submetrics == cedalo_submetrics

print(cirrus_name, "\n")
print(cirrus_submetrics, "\n")

print(cedalo_name, "\n")
print(cedalo_submetrics, "\n")
print(f"Fila {i}")
print("¿Names iguales? →", names_equal)
print("¿Submetrics iguales? →", submetrics_equal)


G52/Snowflake 

[{'dataType': 'DateTime', 'name': 'Timestamp', 'timestamp': 1748235600666, 'value': 1748235600666}] 

G52/Snowflake 

[{'dataType': 'DateTime', 'name': 'Timestamp', 'timestamp': 1748235600666, 'value': 1748235600666}] 

Fila 0
¿Names iguales? → True
¿Submetrics iguales? → True


In [ ]:
# Ver top level
print("Top-level keys:", msg_dict.keys())

# Acceder a los "metrics"
metrics = msg_dict["metrics"]
print("Cantidad de elementos en metrics:", len(metrics))

# Ver el primero de los elementos de "metrics"
first_metric = metrics[2]
print("Primer metric:", first_metric)

# Acceder a los "value" del primer metric
value = first_metric["value"]
print("Keys dentro de value:", value.keys())

# Acceder a los sub-metrics
sub_metrics = value["metrics"]
print("Sub-metrics:", sub_metrics)


# 2do Intento: uso de tuplas y firmas

In [5]:
import pandas as pd
import json

# Cargar los CSVs completos
df_cirrus = pd.read_csv("CirrusLink_10000.csv")
df_cedalo = pd.read_csv("Cedalo_10000.csv")


In [115]:
# Función para CirrusLink
def extract_signature_from_cirrus(row):
    try:
        msg = json.loads(row)
        metric = msg["metrics"][0]
        name = metric["name"]
        submetrics = metric["value"]["metrics"]
        # Creamos una tupla con lo importante
        sub_sig = tuple((m["name"], m["timestamp"], m["value"]) for m in submetrics)
        return (name, sub_sig)
    except Exception as e:
        print("Error en fila Cirrus:", e)
        return None

# Función para Cedalo
def extract_signature_from_cedalo(row):
    try:
        payload = json.loads(row)
        metric = payload[0]
        name = metric["name"]
        submetrics = metric["value"]["metrics"]
        sub_sig = tuple((m["name"], m["timestamp"], m["value"]) for m in submetrics)
        return (name, sub_sig)
    except Exception as e:
        print("Error en fila Cedalo:", e)
        return None


In [116]:
# Aplicar funciones y eliminar filas problemáticas
cirrus_signatures = df_cirrus["MSG"].apply(extract_signature_from_cirrus).dropna()
cedalo_signatures = df_cedalo["PAYLOAD_JSON"].apply(extract_signature_from_cedalo).dropna()

# Convertir a sets
set_cirrus = set(cirrus_signatures)
set_cedalo = set(cedalo_signatures)


In [117]:
# Ver qué firmas de Cirrus no están en Cedalo
missing = set_cirrus - set_cedalo

print(f"Total de firmas únicas en Cirrus: {len(set_cirrus)}")
print(f"Total de firmas únicas en Cedalo: {len(set_cedalo)}")
print(f"Firmas de Cirrus que NO están en Cedalo: {len(missing)}")

# Imprimir algunos ejemplos
if missing:
    print("\nEjemplos de firmas faltantes:")
    for m in list(missing)[:5]:
        print(m)


Total de firmas únicas en Cirrus: 10000
Total de firmas únicas en Cedalo: 10000
Firmas de Cirrus que NO están en Cedalo: 88

Ejemplos de firmas faltantes:
('G201L/Snowflake', (('ProcessVariables/Generator/Rx_618_Generator_Power', 1748241927106, 1157),))
('G902/Snowflake', (('Edge/Motor/Spark_Reference_Cylinder_5', 1748253537328, 113),))
('Compression/Snowflake', (('Edge/Process/LT101_Val', 1748244099728, 12.608835),))
('G902/Snowflake', (('Edge/Generator/Engine_Running_Seconds', 1748247540812, 3433874),))
('G201L/Snowflake', (('ProcessVariables/Generator/Rx_607_Generator_Winding_Temperature_L2', 1748246770072, 72.5),))


In [118]:
# Creamos un dict: firma → índice en Cedalo
cedalo_signature_to_index = {
    sig: idx for idx, sig in cedalo_signatures.items()
}


In [119]:
# Lista para guardar los índices de coincidencia
cirrus_to_cedalo_index = []

for idx, sig in cirrus_signatures.items():
    match_idx = cedalo_signature_to_index.get(sig, None)
    cirrus_to_cedalo_index.append((idx, match_idx))


In [120]:
# Mostrar filas sin match
print("Filas de Cirrus sin match en Cedalo:")
no_match = [(c_idx, cirrus_signatures[c_idx]) for c_idx, match_idx in cirrus_to_cedalo_index if match_idx is None]
print(f"Total: {len(no_match)}")
for c_idx, sig in no_match[:5]:  # Muestra hasta 5
    print(f"Fila Cirrus {c_idx} sin match: {sig}")


Filas de Cirrus sin match en Cedalo:
Total: 88
Fila Cirrus 234 sin match: ('G201L/Snowflake', (('ProcessVariables/Motor/Rx_520_Exhaust_Gas_Temperature_Cylinder_5', 1748236316551, 542),))
Fila Cirrus 685 sin match: ('G53/Snowflake', (('Timestamp', 1748237530927, 1748237530927),))
Fila Cirrus 744 sin match: ('G902/Snowflake', (('Edge/Generator/Engine_Running_Hours', 1748237563099, 1021.4021),))
Fila Cirrus 745 sin match: ('Compression/Snowflake', (('Edge/ACS_Vector_6200/TCV103_Position_Command_Val', 1748237564100, 50.3386),))
Fila Cirrus 908 sin match: ('G201L/Snowflake', (('ProcessVariables/Generator/Rx_604_Generator_Desbalance_Current', 1748237930832, 1.3995444),))


In [121]:
# Crear un DataFrame de mapeo
mapping_df = pd.DataFrame(cirrus_to_cedalo_index, columns=["Cirrus_Index", "Cedalo_Index"])
mapping_df.head(20)


,Cirrus_Index,Cedalo_Index
0,0,2.0
1,1,1.0
2,2,0.0
3,3,5.0
4,4,4.0
5,5,3.0
6,6,7.0
7,7,8.0
8,8,6.0
9,9,11.0


In [122]:
mapping_df[mapping_df["Cirrus_Index"] == 234]


,Cirrus_Index,Cedalo_Index
234,234,NaN


In [123]:
extra_in_cedalo = set_cedalo - set_cirrus
print(f"Firmas de Cedalo que NO están en Cirrus: {len(extra_in_cedalo)}")

if extra_in_cedalo:
    print("\nEjemplos de firmas adicionales en Cedalo:")
    for sig in list(extra_in_cedalo)[:5]:
        print(sig)


Firmas de Cedalo que NO están en Cirrus: 88

Ejemplos de firmas adicionales en Cedalo:
('G52/Snowflake', (('Timestamp', 1748247980290, 1748247980290),))
('Dewpoint/Snowflake', (('Timestamp', 1748254020099, 1748254020074),))
('G201L/Snowflake', (('Timestamp', 1748247082859, 1748247082859),))
('Dewpoint/Snowflake', (('Timestamp', 1748253525385, 1748253525318),))
('G52/Snowflake', (('Timestamp', 1748243541392, 1748243541392),))


In [124]:
cedalo_extra_indices = [
    idx for idx, sig in cedalo_signatures.items() if sig in extra_in_cedalo
]

print(f"\nTotal de filas en Cedalo con firmas exclusivas: {len(cedalo_extra_indices)}")
print("Primeros índices:", cedalo_extra_indices[:5])



Total de filas en Cedalo con firmas exclusivas: 88
Primeros índices: [234, 685, 742, 745, 908]


In [11]:
import pandas as pd
import json

# Cargar los CSVs

df_cirrus = pd.read_csv("CirrusLink_10000.csv")
df_cedalo = pd.read_csv("Cedalo_10000.csv")

def extract_submetrics_cirrus(row):
    try:
        msg = json.loads(row)
        submetrics = []
        for metric in msg.get("metrics", []):
            for m in metric.get("value", {}).get("metrics", []):
                submetrics.append((m["name"], m["timestamp"], m["value"]))
        return submetrics
    except Exception as e:
        print("Error en fila Cirrus:", e)
        return []

def extract_submetrics_cedalo(row):
    try:
        payload = json.loads(row)
        submetrics = []
        for metric in payload:
            for m in metric.get("value", {}).get("metrics", []):
                submetrics.append((m["name"], m["timestamp"], m["value"]))
        return submetrics
    except Exception as e:
        print("Error en fila Cedalo:", e)
        return []


# Extraer todas las submétricas de todas las filas (lista de listas)
cirrus_submetrics_lists = df_cirrus["MSG"].apply(extract_submetrics_cirrus)
cedalo_submetrics_lists = df_cedalo["PAYLOAD_JSON"].apply(extract_submetrics_cedalo)

# Aplanar todas las listas en un solo iterable y convertir a set para unicidad
all_cirrus_submetrics = set([item for sublist in cirrus_submetrics_lists for item in sublist])
all_cedalo_submetrics = set([item for sublist in cedalo_submetrics_lists for item in sublist])

print(f"Total submétricas únicas en Cirrus: {len(all_cirrus_submetrics)}")
print(f"Total submétricas únicas en Cedalo: {len(all_cedalo_submetrics)}")

# Submétricas de Cirrus que no están en Cedalo
missing_submetrics = all_cirrus_submetrics - all_cedalo_submetrics
print(f"Submétricas en Cirrus que NO están en Cedalo: {len(missing_submetrics)}")

if missing_submetrics:
    print("Ejemplos de submétricas faltantes:")
    for i, sm in enumerate(missing_submetrics):
        print(sm)
        if i >= 1000:
            break

# Opcional: Submétricas extra en Cedalo
extra_in_cedalo = all_cedalo_submetrics - all_cirrus_submetrics
print(f"Submétricas en Cedalo que NO están en Cirrus: {len(extra_in_cedalo)}")

if extra_in_cedalo:
    print("Ejemplos de submétricas adicionales en Cedalo:")
    for i, sm in enumerate(extra_in_cedalo):
        print(sm)
        if i >= 9:
            break


Total submétricas únicas en Cirrus: 871596
Total submétricas únicas en Cedalo: 871465
Submétricas en Cirrus que NO están en Cedalo: 131
Ejemplos de submétricas faltantes:
('Timestamp', 1748252674097, 1748252673194)
('Timestamp', 1748250373328, 1748250372984)
('Timestamp', 1748236621729, 1748236620901)
('Timestamp', 1748240862667, 1748240861698)
('Timestamp', 1748249235807, 1748249235444)
('Timestamp', 1748251630817, 1748251630676)
('Timestamp', 1748244079512, 1748244079077)
('Timestamp', 1748253846799, 1748253845832)
('Timestamp', 1748248669105, 1748248668669)
('Timestamp', 1748240799648, 1748240799618)
('Timestamp', 1748247722942, 1748247722267)
('Timestamp', 1748251719997, 1748251719802)
('Timestamp', 1748251287270, 1748251287216)
('Timestamp', 1748253653598, 1748253653568)
('Timestamp', 1748243636624, 1748243636461)
('Timestamp', 1748241357799, 1748241357408)
('Timestamp', 1748249398544, 1748249397662)
('Timestamp', 1748244110714, 1748244110125)
('Timestamp', 1748250010982, 17482500